In [ ]:
import pandas as pd
import sqlalchemy as sa
import json

In [ ]:
def getCreds(filename,subset,defaults={}):
    '''
    This function helps us connect to a database on hadoop2.  The
    userid password information is stored in a local file encoded
    in json format.  
    Parameters:
    filename is where json encoded userid/password information stored
    subset: dictionary key in json file
    defaults: alternative way to connect to database
    Returns: subdictionary based on subset parameter.
    '''
    try:
        with open(filename,'r') as file:
            D = json.load(file)
            file.close()
            if D[subset]:
                return D[subset]
            else:
                return defaults
    except:
        return defaults

def db_setup(userid,password,database):
    '''
    This function connects us to a database using our login credentials. 
    Paramters:
    userid - string with our userid
    password - string with our password
    database - string with database name
    Returns:
    SQL engine and SQL connection objects
    '''
    template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/{}'
    cstring = template.format(userid,password,database)
    engine = sa.create_engine(cstring)
    connection = engine.connect()
    
    return engine, connection

def SQLtoPandas(sqlQuery):
    '''
    
    
    
    '''
    queryProxy = connection.execute(sqlQuery)
    pandasDf = pd.DataFrame(queryProxy.fetchall(), columns = queryProxy.keys())
    return pandasDf

In [ ]:
#creds = getCreds('creds.json','mysql',defaults = {'user':'studen_j1', 'password':'studen_j1'})
creds = {'user': 'brown_b1', 'password': 'brown_b1'}
try:
    connection.close()
    del engine
except:
    pass
engine, connection = db_setup(creds['user'],creds['password'], "brown_b1")

In [ ]:
PopQuery = '''
SELECT 
s.State, SUM(cp.Population) AS StatePopulation2016,
ROUND(SUM(cp.Population)/(SELECT SUM(cp.Population) FROM CountyPopulations AS cp)*100, 2) AS Percentage 
FROM CountyPopulations AS cp
INNER JOIN States AS s ON s.StateID = cp.StateID
GROUP BY cp.StateID
ORDER BY cp.StateID;
'''

UserPopQuery = '''
SELECT s.State, COUNT(u.MemberID) AS UserPop,
ROUND(COUNT(u.MemberID)/(SELECT COUNT(u.MemberID) FROM Users AS u)*100, 2) AS Percentage 
FROM Users AS u
INNER JOIN States AS s ON s.StateID = u.StateID
WHERE s.State IN ('VA','MD','DC')
GROUP BY u.StateID
ORDER BY u.StateID;
'''

UserDemographicQuery = '''
SELECT u.MemberID, u.UserLongitude, u.UserLatitude, u.UserCity, s.State
FROM Users AS u
INNER JOIN States AS s ON s.stateID = u.stateID
WHERE s.State IN ('VA','MD','DC');
'''

comparisonQuery = '''
SELECT s.State, actual.pop AS StatePopulation2016,actual.percentage AS ActualPct, api.pop AS UserPop,api.percentage AS MeetUpPct FROM States as s
INNER JOIN (SELECT s.StateID,SUM(cp.Population) AS pop,ROUND(SUM(cp.Population)/(SELECT SUM(cp.Population) FROM CountyPopulations AS cp)*100, 2) 
AS Percentage FROM CountyPopulations as cp
INNER JOIN States AS s ON s.StateID = cp.StateID
GROUP BY cp.StateID
ORDER BY cp.StateID)
AS actual ON s.StateID = actual.StateID
INNER JOIN (SELECT s.StateID,COUNT(u.MemberID) AS pop,ROUND(COUNT(u.MemberID)/(SELECT COUNT(u.MemberID) FROM Users AS u)*100, 2)
AS Percentage FROM Users AS u
INNER JOIN States AS s ON s.StateID = u.StateID
GROUP BY u.StateID
ORDER BY u.StateID) 
AS api ON s.StateID = api.StateID
ORDER BY s.StateID;
'''

actualPopDf = SQLtoPandas(PopQuery)
UserPopDf = SQLtoPandas(UserPopQuery)
UserDemographicDf = SQLtoPandas(UserDemographicQuery)
comparisonDf = SQLtoPandas(comparisonQuery)
pop = comparisonDf.melt(id_vars='State',value_vars=['StatePopulation2016','UserPop'],var_name='Type',value_name='Population')
pct = comparisonDf.melt(id_vars='State',value_vars=['ActualPct','MeetUpPct'],var_name='Type',value_name='Percentage')
for index,rows in pop.iterrows():
    if rows['Type'] == 'StatePopulation2016':
        pop.loc[[index],['Type']] = 'Actual'
    else:
        pop.loc[[index],['Type']] = 'Meetup'      
for index,rows in pct.iterrows():
    if rows['Type'] == 'ActualPct':
        pct.loc[[index],['Type']] = 'Actual'
    else:
        pct.loc[[index],['Type']] = 'Meetup'
comparisonDf = pd.merge(pop, pct, on=['State','Type'])

UserPopDf.to_csv('UserPopDf.csv')
UserDemographicDf.to_csv("UserDemographicDf.csv")

In [ ]:
print(actualPopDf)
print(UserPopDf)
print(UserDemographicDf)
print(comparisonDf)

In [ ]:
try:
    connection.close()
    del engine
    print("Connection closed")
except:
    pass
    print("No connection")